# VacationPy
--- 
**Brief Description**: This notebook reflects the analysis of the dataset generated by *WeatherPy* challenge to plan an ideal vacation destination. The entire report is organized and presented using heatmaps by ingesting data from a public API. 
***
> - **Requirement 1:** Create a city heatmap with humidity as the key metric to indicate the relative sizing. This heatmap is then used to narrow down the list of cities based on the following ideal weather conditions for planning the vacation.<br>
       1. Maximum temperature range is between 65 degrees F to 80 degrees F <br>
       2. There is no cloudiness.<br>
       3. Wind Speed is less than 10 miles per hour (mph) <br>
       
> - **Requirement 2:** Nearest hotel for vacation stay based on the above: <br>
       1. Using a public API, the nearest hotel to a particular city of interest, based on the above list is found.<br> 
       2. An updated heatmap that shows the hotel of interest is presented.<br>
***

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews

# Define a dictionary structure for header row styling
df_header_row = {
    'selector': 'th.col_heading',
    'props': 'background-color: bisque;'
}

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data - minor data wrangling for hiding the index
city_data_df.head().style.hide(axis="index").set_table_styles([df_header_row])

City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cabo San Lucas,22.890900,-109.912400,77.310000,50,20,19.570000,MX,1707345290
1,Novodvinsk,64.416500,40.812200,-20.220000,100,99,3.690000,RU,1707345290
2,Constanța,44.183300,28.650000,45.280000,68,100,13.330000,RO,1707345245
3,New Norfolk,-42.782600,147.058700,66.700000,54,63,3.180000,AU,1707345291
4,George Town,5.411200,100.335400,80.530000,93,20,3.440000,MY,1707345292


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot using a combination of hvplot pandas and geoviews custom parameters
city_humidity_map = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500, scale = 1.20, title = "Humidity Heat Map", size = "Humidity", color = "City")

# Display the map
city_humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

# Step 1: Define ideal weather conditions to shortlist cities for a vacation stay
# Max Temperature Range: 65 degrees to 80 degrees F, Wind Speed: less than 10 mph, Cloudiness is zero
# Drop any rows with null values
ideal_weather_condition_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) 
                                             &(city_data_df["Max Temp"] > 65) 
                                             &(city_data_df["Wind Speed"] < 10) 
                                             &(city_data_df["Cloudiness"] == 0),:].dropna()
# Display sample data
ideal_weather_condition_df.head().style.hide(axis="index").set_table_styles([df_header_row])

City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,Margaret River,-33.950000,115.066700,67.510000,46,0,3.760000,AU,1707345112
211,East London,-33.015300,27.911600,68.630000,61,0,8.140000,ZA,1707345225
238,Mar del Plata,-38.002300,-57.557500,77.020000,73,0,3.440000,AR,1707345413
294,Kīsh,26.557800,54.019400,66.470000,63,0,9.220000,IR,1707345439
356,Beaufort West,-32.356700,22.583000,68.520000,51,0,7.180000,ZA,1707345469


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_condition_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head().style.hide(axis="index").set_table_styles([df_header_row])

City,Country,Lat,Lng,Humidity,Hotel Name
Margaret River,AU,-33.950000,115.066700,46,
East London,ZA,-33.015300,27.911600,61,
Mar del Plata,AR,-38.002300,-57.557500,73,
Kīsh,IR,26.557800,54.019400,63,
Beaufort West,ZA,-32.356700,22.583000,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Step 1: Set parameters to search for a hotel
limit = 20
radius = 10000

# Step 2: Define the params dictionary data structure to build the API URL for the search based on GeoAPI specifications
params = {
        "limit":limit,
        "apiKey":geoapify_key,
        "categories":"accommodation.hotel"
         }

# Step 3: Print a message to follow up the hotel search
print("Starting hotel search")

# Step 4: Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Step 5: Display sample data
hotel_df.head(10).style.hide(axis="index").set_table_styles([df_header_row])

Starting hotel search
Margaret River - nearest hotel: Margaret River Hotel
East London - nearest hotel: No hotel found
Mar del Plata - nearest hotel: Gran hotel Mónaco
Kīsh - nearest hotel: هتل فارابی
Beaufort West - nearest hotel: Matoppo Inn
Irymple - nearest hotel: Chaffey Motor Inn
Tabuk - nearest hotel: Davidson Hotel
Minbu - nearest hotel: Rolex Guesthouse
Zouérat - nearest hotel: فندق تيرس زمور
Maun - nearest hotel: Center Lodge Conference Center


City,Country,Lat,Lng,Humidity,Hotel Name
Margaret River,AU,-33.950000,115.066700,46,Margaret River Hotel
East London,ZA,-33.015300,27.911600,61,No hotel found
Mar del Plata,AR,-38.002300,-57.557500,73,Gran hotel Mónaco
Kīsh,IR,26.557800,54.019400,63,هتل فارابی
Beaufort West,ZA,-32.356700,22.583000,51,Matoppo Inn
Irymple,AU,-34.250000,142.166700,59,Chaffey Motor Inn
Tabuk,PH,17.418900,121.444300,85,Davidson Hotel
Minbu,MM,20.183300,94.883300,80,Rolex Guesthouse
Zouérat,MR,22.718700,-12.452100,10,فندق تيرس زمور
Maun,BW,-19.983300,23.416700,65,Center Lodge Conference Center


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot using a combination of hvplot pandas and geoviews custom parameters
city_humidity_map_enhanced = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500, scale = 1.20, title = "Humidity Heat Map (with hover)", hover_cols = ["Country","Hotel Name"], size = "Humidity", color = "City")

# Display the map
city_humidity_map_enhanced

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)